In [1]:
from ChainCodes import *

import numpy as np
from tqdm import tqdm
from multiprocessing import Pool

# Error params

In [2]:
T1 = 30 # Qubit relaxation time
Tf = 60 # Qubit dephasing time
Tg1Q = 0.02 # Single-qubit gate time
Tg2Q = 0.04 # Two-qubit gate time
tm = 0.3 # Measurement time
td = 0.3 # Depletion time


p_axis = 10**(-4) # In-axis rotation error
p_plane = 5 * 10**(-4) # In-plane rotation error 

dphi = 0.01

eta_origin = np.array([[0.9985, 0, 0.0015, 0],
                       [0.005, 0.0015, 0.0149, 0.9788]])

eta = np.zeros((2, 3))
for i in range(0, 3):
    eta[:, i] = np.sum(eta_origin[:, :i+1], axis = 1)
eta


#Hadamard_noise:
ph = (1-np.exp(dphi**2/2))/(2-np.sqrt(2))
# ph = 0

# ZZ_noise
pZZ = (1-np.exp(dphi**2/2))/2

#measure
p0 = eta_origin[0, 2] + eta_origin[0, 3]
p1 = eta_origin[1, 0] + eta_origin[1, 1]



# Tg1Q = 0.0 # Single-qubit gate time
# Tg2Q = 0.0 # Two-qubit gate time
# tm = 0 # Measurement time
# td = 0 # Depletion time

# p_axis = 0
# p_plane = 0

# #Hadamard_noise:
# ph = 0

# # ZZ_noise
# pZZ = 0

# #measure
# p0 = 0
# p1 = 0

arg = {
    'T1':        T1,
    'Tf':        Tf,
    'Tg1Q':      Tg1Q,
    'Tg2Q':      Tg2Q,
    'tm':        tm,
    'td':        td,
    'p_axis':    p_axis,
    'p_plane':   p_plane,
    'ph':        ph,
    'pZZ':       pZZ,
    'p0':        p0,
    'p1':        p1
}

CC = ChainCodes(arg)

# Run

In [6]:
MultiProcessing = True
N_process = 32
N = 10
fout = '_stab_instate'
N_run = 1000000
N_cycle = 20
M_datanomeas = np.zeros((N_run, N_cycle, N//2), dtype = np.int8)
M_ancillanomeas = np.zeros((N_run, N_cycle, N//2), dtype = np.int8)
basis = '0'
table = init_state0(N)
odd = np.arange(N//2)*2 + 1
Meas = []


if basis == '1':
    for i in odd:
        table = X_gate(table, i)
elif basis == '+':
    for i in odd:
        table = Ry05_gate(table, i)
elif basis == '-':
    for i in odd:
        table = Ry_05_gate(table, i)
elif basis == 'i+':
    for i in odd:
        table = Rx_05_gate(table, i)
elif basis == 'i-':
    for i in odd:
        table = Rx05_gate(table, i)

CC.set_full_Cycle(table, N, N_cycle, basis)
if MultiProcessing:
    with Pool(N_process) as p:
        Meas = list(tqdm(p.imap(CC.full_Cycle, range(N_run)), total=N_run))
else:
    Meas = list(tqdm(map(CC.full_Cycle, range(N_run)), total=N_run))

M_datanomeas = np.asarray(Meas, dtype=np.int8)[:,0]
M_ancillanomeas = np.asarray(Meas, dtype=np.int8)[:,1]


M_data = add_noise_to_measurements(M_datanomeas, p0, p1)
M_ancilla = add_noise_to_measurements(M_ancillanomeas, p0, p1)


for i in tqdm(range(N_run)):
    for j in range(N_cycle):
        M_ancillanomeas[i, j] = np.roll(M_ancillanomeas[i, j], 2*j)
        M_datanomeas[i, j] = np.roll(M_datanomeas[i, j], -2*j)
        M_ancilla[i, j] = np.roll(M_ancilla[i, j], 2*j)
        M_data[i, j] = np.roll(M_data[i, j], -2*j)

        
np.savez_compressed('N' + str(N) + fout + basis + '_' + str(N_run), M_data = M_data, M_ancilla = M_ancilla,\
                           M_datanomeas = M_datanomeas, M_ancillanomeas = M_ancillanomeas)

100%|██████████████████████████████████████████████████████████████████████| 1000000/1000000 [15:21<00:00, 1084.73it/s]
